In [129]:
import pandas as pd
import numpy as np
from sklearn import linear_model
pd.options.display.max_columns = None
pd.options.display.max_rows = None

# Import data

In [130]:
# read in data
# file34 = 'SI Table S3-4_Diesel Vehicles-Zhao et al 2015.xlsx'
file15 = 'SI-Table S1,S5-PNAS.xlsx'
file24 = 'Supporting Information-Tables S2-S4_PNAS.xlsx'
fileMW = 'speciation_MW.xlsx'
# put data into pandas
# SI34 = pd.ExcelFile(file34)
SI15 = pd.ExcelFile(file15)
SI24 = pd.ExcelFile(file24)
# Put the MW into different variables
SIMW = pd.ExcelFile(fileMW)

# Parse the sheets in each file into separate dataframes

In [131]:
# There appears to be multiple S3 and S4 files, though all labeled differently. 
# The files I have are: S1, S2, S3, S3a, S3b, S4, S4a, S4b, S5
# It appears that S1, S2, S3a, S3b, S4, and S5 are part of one set of experiments
# and S3, S4a, S4b are part of another.

print(SI15.sheet_names)
# Read in data
df_S1 = pd.DataFrame(SI15.parse('Table S1'))
df_S5 = pd.DataFrame(SI15.parse('Table S5'))

print(SI24.sheet_names)
# Read in data
df_S2  = pd.DataFrame(SI24.parse('Table S2')).dropna(axis=0)
df_S3a = pd.DataFrame(SI24.parse('Table S3a'))
df_S3b = pd.DataFrame(SI24.parse('Table S3b'))
df_S4  = pd.DataFrame(SI24.parse('Table S4'))

# some processing into uniform units (mg/kg-fuel)
df_S3a.iloc[:,1:] = df_S3a.iloc[:,1:] / 1000 # change from ug/kg-fuel to mg/kg-fuel
# make all labels for ID the same and sort them by test ID value
df_S2  = df_S2.rename(index=str, columns={"TEST ID NUM": "test_ID"}).sort_values(by='test_ID')
df_S3a = df_S3a.rename(index=str, columns={"TEST ID": "test_ID"}).sort_values(by='test_ID')
df_S3b = df_S3b.rename(index=str, columns={"TEST ID": "test_ID"}).sort_values(by='test_ID')
df_S4  = df_S4.rename(index=str, columns={"Test ID": "test_ID"}).sort_values(by='test_ID')
df_S5  = df_S5.rename(index=str, columns={"Test_ID_table5": "test_ID"}).sort_values(by='test_ID')

# Parse the MW data
print(SIMW.sheet_names)
S2_MW = pd.DataFrame(SIMW.parse('S2_MW')).set_index('Species').T
S3a_MW = pd.DataFrame(SIMW.parse('S3a_MW')).set_index('Species').T
S3b_MW = pd.DataFrame(SIMW.parse('S3b_MW')).set_index('Species').T
S4_MW = pd.DataFrame(SIMW.parse('S4_MW')).set_index('Species').T
S3_MW = pd.concat([S3a_MW, S3b_MW], axis=1)

['Table S1', 'Table S5']
['Table S2', 'Table S3a', 'Table S3b', 'Table S4']
['S2_MW', 'S3a_MW', 'S3b_MW', 'S4_MW']


# Pre-processing

Merge tables together based on their test ID numbers

In [132]:
# This has the secondary effect of finding which experiments have incomplete data 

# merge 3a and 3b together because they share all rows with each other
df_S3 = pd.merge(df_S3a, df_S3b, how='inner', on='test_ID')

# first, get the inner join of S5 and S2, then use that to pull out the rows in S5 not in S2, and the rows in S2 not in S5.
df_52 = pd.merge(df_S5, df_S2, how='inner', on='test_ID')
# then, find the rows that are in 5 but not in 2
df_5_noVOC = pd.merge(df_S5, df_52, how='left', indicator=True) \
                .query("_merge == 'left_only' ") \
                .drop('_merge',1)
df_5_noVOC = df_5_noVOC.dropna(axis=1, how='all')

# ok, so we now have two dataframes: df_52, and df_5_no2. Let's see which of these has data in 3.
# we should get out 4 dataframes: df_523, a dataframe with complete data, df_52_no3, and df_5_no2_3, and df_5_no2_no3
df_523 = pd.merge(df_52, df_S3, how='inner', on='test_ID')

# then, find the rows that are in 52 but not in 3
df_52_noIVOC = pd.merge(df_52, df_523, how='left', indicator=True) \
                .query("_merge == 'left_only' ") \
                .drop('_merge', 1)
df_52_noIVOC = df_52_noIVOC.dropna(axis=1, how='all')

# then, find the rows that are in 5 and 3, but not 2
# code works, but it's an empty matrix
# df_5_no2_3 = pd.merge(df_5_no2, df_S3, how='inner', on='test_ID')

# therefore, df_5_noVOC_noIVOC is the same as df_5_noVOC
df_5_noVOC_noIVOC = df_5_noVOC

# ok, now we have 3 dataframes: df_523, df_52_no3, and df_5_no2_no3. Let's see which of these has data in 4.
# we should get out 6 dataframes: df_5234, df_523_no4, df_52_no3_4, df_52_no3_no4, df_5_no2_no3_4, df_5_no2_no3_no4
# though some of these may turn out to be empty

# first, find the experiments with all the data
df_5234 = pd.merge(df_523, df_S4, how='inner', on='test_ID')

# then, find the rows that are in 523, but not 4
df_523_noSVOC = pd.merge(df_523, df_S4, how='left', indicator=True) \
                .query("_merge == 'left_only' ") \
                .drop('_merge', 1)
df_523_noSVOC = df_523_noSVOC.dropna(axis=1, how='all')

# then find the exps with data in 5, 2, and 4, but not 3
# code works, but it's empty
# df_52_no3_4 = pd.merge(df_52_no3, df_S4, how='inner', on='test_ID')
# this means that df_52_no3 is the same as df_52_no3_no4
df_52_noIVOC_noSVOC = df_52_noIVOC

# now find the exps with data in 5 and 4, but not in 2 or 3
# df_5_no2_no3_4 = pd.merge(df_5_no2_no3, df_S4, how='inner', on='test_ID')
# also empty.
# so df_5_no2_no3_no4 is the same as df_5_no2_no3
df_5_nodata = df_5_noVOC_noIVOC

# so, finally, we have 4 different groups:
# df_5234, the experiments with data on VOCs, IVOCs, and SVOCs
# df_523_noSVOC, the experiments with data on VOCs and IVOCs, but not SVOCs
# df_52_noIVOC_noSVOC, the experiments with data on VOCs, but not IVOCs or SVOCs
# df_5_noVOC_noIVOC_noSVOC, the experiments with no data on VOCs, IVOCs, or SVOCs

In [133]:
# Find out how many experiments there are in each group
print(df_5234.shape) # All data available, VOC, IVOC, SVOC
print(df_523_noSVOC.shape) # Has VOC, IVOC, missing SVOC
print(df_52_noIVOC_noSVOC.shape) # Has VOC, missing IVOC, SVOC
print(df_5_nodata.shape) # Has no data beyond experiment details
print(df_S5.shape) # Total number of experiments with details

(11, 301)
(5, 291)
(12, 212)
(5, 9)
(33, 9)


Read the category files

In [134]:
# This is easiest done to the original sheets, where the speciations are all separate
# import all the category data from excel sheets. 
# Any changes to species categorization must be made in the excel sheet
fileCat = 'speciation_SOMcategories.xlsx' # file name
speciesCat = pd.ExcelFile(fileCat) # import file
print(speciesCat.sheet_names)

# Read the files from 
S2_cat = pd.DataFrame(speciesCat.parse('S2_categories'))
S3a_cat = pd.DataFrame(speciesCat.parse('S3a_categories'))
S3b_cat = pd.DataFrame(speciesCat.parse('S3b_categories'))
S4_cat = pd.DataFrame(speciesCat.parse('S4_categories'))

['S2_categories', 'S3a_categories', 'S3b_categories', 'S4_categories']


## Processing

The text is unclear on whether correlation for missing IVOCs is based on NMOG-EF or NMOG-ppb.  
If it is based on NMOG-EF, then it should be calculated before mass-to-volume-basis change calculations.  
Here, we attempt to calculate based on NMOG-EF.

In [135]:
# Use all experiments with NMOG-EF data and IVOC data 
df_S23 = pd.merge(df_S2, df_S3, how='inner', on='test_ID')
S23_NMOGEF = df_S23.pop('NMOG(mg/kg-fuel)')
# sum the IVOC-EFs over each experiment
totalEF_IVOC = df_S23.loc[:,'Dodecane':'Unspeciated cyclic compound Bin22'].sum(axis=1)

# Run a linear model here
regr = linear_model.LinearRegression(fit_intercept=False)
X = pd.DataFrame(S23_NMOGEF)
y = pd.DataFrame(totalEF_IVOC).rename(index=str, columns={0:'IVOC-EF'})
regr.fit(X, y)
print('R^2 is ' + str(regr.score(X,y)))
print('Slope is ' + str(regr.coef_[0][0]))
print('Intercept is ' + str(regr.intercept_))

R^2 is 0.920039160287
Slope is 0.0382088091432
Intercept is 0.0


These results match very well with Yunliang's reported numbers.  
Now we create a normalized average profile of IVOCs and multiply them by the NMOG-EF.

In [136]:
IVOC_EF = df_S23.loc[:,'Dodecane':'Unspeciated cyclic compound Bin22'].div(S23_NMOGEF, axis=0)
IVOC_EF_avg = IVOC_EF.mean(axis=0)
IVOC_EF_normalized = IVOC_EF_avg.div(IVOC_EF_avg.sum()) # Series
IVOC_EF_normalized

# Select the exps with missing IVOCs
df_S2_no3 = pd.merge(df_S2, df_S3, how='left', on='test_ID', indicator=True) \
            .query(" _merge == 'left_only' ") \
            .drop('_merge',1)
S2_no3_NMOG = df_S2_no3.loc[:,'NMOG(mg/kg-fuel)'] # Series

# Multiply the two series together to make a matrix
# See https://stackoverflow.com/questions/19570693/creating-a-dataframe-in-pandas-by-multiplying-two-series-together/29686743#29686743
# for why the follow command was written this way
dftemp = pd.DataFrame(1, index=S2_no3_NMOG.index, columns=IVOC_EF_normalized.index)
missingIVOCs = dftemp.multiply(S2_no3_NMOG, axis='index') * IVOC_EF_normalized * regr.coef_[0][0]
# Plop the missing values back into the dataframe
df_S2_added3 = df_S2_no3.fillna(missingIVOCs)

# Another 11 experiments can now be run with IVOCs
df_S52_added3 = pd.merge(df_S5, df_S2_added3, how='inner', on='test_ID')

In [137]:
# SVOCs, if they ever get done, would be placed here.

### Calculate the average MW of an average mixture of organics

In [138]:
# Based on 
# pop out the NMOG concentrations so that we can divide everything by MW, and then NMOG
NMOG_5234 = df_5234.pop('NMOG(mg/kg-fuel)') 

In [139]:
MW_5234 = pd.concat([S2_MW, S3a_MW, S3b_MW, S4_MW], axis=1, sort=False)
first_species_5234 = MW_5234.columns.values[0]
last_species_5234 = MW_5234.columns.values[-1]
index_first_5234 = df_5234.columns.get_loc(first_species_5234)
index_last_5234  = df_5234.columns.get_loc(last_species_5234)

# Turning back to the data
# Separate the experiment info and the species data
info_5234 = df_5234.iloc[:, :index_first_5234]
species_5234 = df_5234.iloc[:, index_first_5234:index_last_5234]
# Divide the concentrations by the MW
temp = species_5234.divide(MW_5234.iloc[0,:], axis='columns')
# Then divide by the NMOG
temp2 = temp.div(NMOG_5234, axis='rows')
# Find the median for each of the species
# *** This is currently the median for all types of vehicles, but ideally we would separate them.
# *** Unfortunately, there aren't many samples here, so we'll have to figure out how to handle that
tempMedian = temp2.median(axis='rows')
fracMedian = tempMedian / tempMedian.sum()

fracMW = MW_5234.mul(fracMedian, axis='columns')
# sum up everything in each row and divide by the sum
avgMW = fracMW.sum(axis='columns')
avgMW[0]

63.999053896711644

### Use the average MW to transform the mass-basis concentrations to a volume(mole)-basis

We focus first on df_S52_added3, the dataframe with inferred IVOCs

In [140]:
df_S52_added3 = pd.merge(df_S5, df_S2_added3, how='inner', on='test_ID')
NMOG_EF_52a3 = df_S52_added3.pop('NMOG(mg/kg-fuel)')
NMOG_ppb_52a3 = df_S52_added3.loc[:,'NMOG (ppb)']

# Take care of the VOCs first
df_S52_added3_copy = df_S52_added3.copy()
## Divide by each species' molecular weight, then multiply by the average molecular weight
df_S52_added3_copy.loc[:,'formaldehyde':'n-dodecane'] = df_S52_added3.loc[:,'formaldehyde':'n-dodecane'] \
                                                    .divide(S2_MW.iloc[0], axis='columns')*avgMW[0]
## Then divide by the overall NMOG EF, and multiply by the overall NMOG ppb, to get VOCs in ppb                                            
df_S52_added3_copy.loc[:,'formaldehyde':'n-dodecane'] = df_S52_added3_copy.loc[:,'formaldehyde':'n-dodecane'] \
                                                        .div(NMOG_EF_52a3, axis='index') \
                                                        .mul(NMOG_ppb_52a3, axis='index')

# Then take care of the IVOCs
## Divide by each species' molecular weight, then multiply by the average molecular weight
df_S52_added3_copy.loc[:,'Dodecane':'Unspeciated cyclic compound Bin22'] \
                    = df_S52_added3.loc[:,'Dodecane':'Unspeciated cyclic compound Bin22'] \
                        .divide(S3_MW.iloc[0], axis='columns')*avgMW[0]
## Then divide by the overall NMOG EF, and multiply by the overall NMOG ppb, to get IVOCs in ppb
df_S52_added3_copy.loc[:,'Dodecane':'Unspeciated cyclic compound Bin22'] = df_S52_added3_copy.loc[:,'Dodecane':'Unspeciated cyclic compound Bin22'] \
                                                        .div(NMOG_EF_52a3, axis='index') \
                                                        .mul(NMOG_ppb_52a3, axis='index')

df_VOC_addedIVOC = pd.concat([df_S52_added3_copy.loc[:,'test_ID'], df_S52_added3_copy.loc[:,'formaldehyde':'Unspeciated cyclic compound Bin22']], axis='columns')

In [141]:
# Now that everything is in ppb, we can categorize and put it into a format for SOM

# create a list of group categories to pass into the parsing function
categories = ['isoprene', 'aPinene', 'sesq', 'benzene', 'toluene', 'xylenes', 'naphthalene', 'n-alkane', 'b-alkane', 'c-alkane']
cat_VOC_IVOC = pd.concat([S2_cat, S3a_cat, S3b_cat]).reset_index(drop=True)
# I want a list of reversed numbers to make tranposing them easier later
carbon_num = [x+1 for x in list(reversed(range(23)))] # create a list from 23 to 1, to represent carbon numbers
# Create a list with a dataframe inside it 
list_testID_VOCaIVOC = [pd.DataFrame(df_VOC_addedIVOC.T.iloc[0].astype(int))]

# Now we loop through all the categories, and if the category is alkanes,
# we loop through the carbon numbers.

# for j in range(len(categories)):
#     # Check to see if you're doing alkanes. If you are, you have to loop through carbon numbers too
#     if  (  (categories[j] == 'n-alkane')
#          | (categories[j] == 'b-alkane')
#          | (categories[j] == 'c-alkane')
#         ):
#         for k in range(len(carbon_num)):
#             # This pulls out indices for species that match each alkane type and carbon number
#             # The category matrix is missing the test_ID column though
#             # , so all the indices are offset from the database columns by 1 
#             temp_index_list = cat_VOC_IVOC.index[ (cat_VOC_IVOC['Category']==categories[j]) 
#                                                  & (cat_VOC_IVOC['Carbon number']==carbon_num[k]) ].tolist()
#             # Now we find and sum up the species
#             # Remember, we need the +1 in the data_files indices to account for the test_ID column
#             data_index_list = [x+1 for x in temp_index_list]
#             temp_conc = df_VOC_addedIVOC.iloc[:,(data_index_list)].sum(axis=1)
#             # Store it back into the dataframe
#             list_testID_VOCaIVOC[0][categories[j],carbon_num[k]] = temp_conc                
#     else:            
#         # This pulls out the indices for a category, in a file
#         temp_index_list = cat_VOC_IVOC.index[cat_VOC_IVOC['Category'] == categories[j]].tolist()
#         # This should also require +1 to all the indices because of the test_ID column
#         data_index_list = [x+1 for x in temp_index_list]
#         # This pulls out, then sums, the concentrations matching the categories
#         temp_conc = df_VOC_addedIVOC.iloc[:,data_index_list].sum(axis=1)
#         # Store it in a new dataframe for categorized data
#         list_testID_VOCaIVOC[0][categories[j]] = temp_conc
            
for j in range(len(categories)):
    for k in range(len(carbon_num)):
            # This pulls out indices for species that match each alkane type and carbon number
            # The category matrix is missing the test_ID column though
            # , so all the indices are offset from the database columns by 1 
            temp_index_list = cat_VOC_IVOC.index[ (cat_VOC_IVOC['Category']==categories[j]) 
                                                 & (cat_VOC_IVOC['Carbon number']==carbon_num[k]) ].tolist()
            # Now we find and sum up the species
            # Remember, we need the +1 in the data_files indices to account for the test_ID column
            data_index_list = [x+1 for x in temp_index_list]
            temp_conc = df_VOC_addedIVOC.iloc[:,(data_index_list)].sum(axis=1)
            # Store it back into the dataframe
            list_testID_VOCaIVOC[0][categories[j],carbon_num[k]] = temp_conc           

In [142]:
df_catd_VOC_aIVOC = list_testID_VOCaIVOC[0].reset_index(drop=True)
df_catd_VOC_aIVOC.reset_index(drop=True)
label_names_aIVOC = list(df_catd_VOC_aIVOC)
# alkane_list = ['n-alkane', 'b-alkane', 'c-alkane']
min_index_aIVOC = df_catd_VOC_aIVOC.shape[1]

for k in range(df_catd_VOC_aIVOC.shape[0]):
    exp_row_aIVOC = pd.DataFrame(df_catd_VOC_aIVOC.iloc[k])

    # Initialize an array to store everything
    df_SOM_ready_aIVOC = pd.DataFrame()
    # Loop through the alkanes
    for j in range(len(categories)):
        # Get the indices for the alkanes
        indices_aIVOC = [i for i, elem in enumerate(label_names_aIVOC) if categories[j] in elem] 
        # Get the alkane concentrations themselves (in a column)
        species_aIVOC = exp_row_aIVOC.iloc[indices_aIVOC].reset_index(drop=True)
        species_aIVOC = species_aIVOC.rename(index=str, columns={k:categories[j]})
        # Concat this together with the entire array
        df_SOM_ready_aIVOC = pd.concat([df_SOM_ready_aIVOC, species_aIVOC], axis=1, sort=False)
        # change minimum to the smallest alkane
        if min(indices_aIVOC) < min_index_aIVOC:
            min_index_aIVOC = min(indices_aIVOC)

#     # Get the remaining list of organics
#     df_non_species_aIVOC = exp_row_aIVOC[1:min_index_aIVOC].T.reset_index(drop=True)
#     df_SOM_ready_aIVOC = df_SOM_ready_aIVOC.reset_index(drop=True)

#     # Combine them together into one dataframe    
#     df_SOM_ready_aIVOC = pd.concat([df_non_species_aIVOC, df_SOM_ready_aIVOC], axis=1, sort=False)

    # Replace all the NaN with 0s
    df_SOM_ready_aIVOC = df_SOM_ready_aIVOC.fillna(0)
    # Change the concentration from ppb to ppm
    df_SOM_ready_aIVOC = df_SOM_ready_aIVOC / 1000

    # Give a name to the final dataframe
    df_SOM_ready_aIVOC.name = str(exp_row_aIVOC.loc['test_ID'].iloc[0].astype(int))
    # Save it into a file, using its name
    df_SOM_ready_aIVOC.to_csv(df_SOM_ready_aIVOC.name + '_aIVOC_ppm' + '.csv')

This is the end of filling the missing IVOC data

In [143]:
# Now that we have the average MW, we can turn back to calculating the ppb of precursors
# *** There must be a better way of doing this, will think about it later

In [144]:
# Transform EFs to a volume fraction 

# Because EFs are given as a mass, we need to transform everything to a volume.
# This means dividing by a species' molecular weight and multiplying by the average
# molecular weight. 
# These values will still need to be divided by the total EF,
# and multiplied by the total NMOG in the chamber.
# That will result in the ppb of organic species in the chamber.
df_S2.iloc[:,1:S2_MW.shape[1]]  = df_S2.iloc[:,1:S2_MW.shape[1]].divide(S2_MW.iloc[0], axis='columns')*avgMW[0]
df_S3a.iloc[:,1:S3a_MW.shape[1]] = df_S3a.iloc[:,1:S3a_MW.shape[1]].divide(S3a_MW.iloc[0], axis='columns')*avgMW[0]
df_S3b.iloc[:,1:S3b_MW.shape[1]] = df_S3b.iloc[:,1:S3b_MW.shape[1]].divide(S3b_MW.iloc[0], axis='columns')*avgMW[0]
df_S4.iloc[:,1:S4_MW.shape[1]]  = df_S4.iloc[:,1:S4_MW.shape[1]].divide(S4_MW.iloc[0], axis='columns')*avgMW[0]

In [145]:
# Get some important information out from the original data files, especially the test IDs
# We'll use test IDs later to merge the data sheets together
df_S2_dup = df_S2.copy()
df_S3_dup = pd.merge(df_S3a, df_S3b, how='inner', on='test_ID').copy()
df_S5_dup = df_S5.copy()
NMOG_EF = df_S2_dup.pop('NMOG(mg/kg-fuel)')
NMOG_ppb = df_S5_dup.pop('NMOG (ppb)')
S2_test_ID = df_S2.T.iloc[0].astype(int)
S3a_test_ID = df_S3a.T.iloc[0].astype(int)
S3b_test_ID = df_S3b.T.iloc[0].astype(int)
S4_test_ID = df_S4.T.iloc[0].astype(int)
S5_test_ID = df_S5.T.iloc[0].astype(int)

In [146]:
# attach test IDs to NMOG_EF
df_NMOG_EF  = pd.concat([pd.DataFrame(S2_test_ID), pd.DataFrame(NMOG_EF)], axis=1)
df_NMOG_ppb = pd.concat([pd.DataFrame(S5_test_ID), pd.DataFrame(NMOG_ppb)], axis=1)
# We already divided by MWi and multiplied by avgMW. Now we have to divide by NMOG_EF and multiply by NMOG_ppb

#######
# Filter for common rows between exp info (5) and VOCs (2)
df_S52_test_ID = pd.DataFrame(S5_test_ID).merge(pd.DataFrame(S2_test_ID), how='inner',on='test_ID')
df_NMOG_EF_fil52 = df_NMOG_EF.merge(df_S52_test_ID, how='inner', on='test_ID')
df_NMOG_ppb_fil52 = df_NMOG_ppb.merge(df_S52_test_ID, how='inner', on='test_ID') 
df_data_VOC = df_S2_dup.merge(df_S52_test_ID, how='inner', on='test_ID')
# Divide by NMOG_EF and multiply by NMOG ppb
df_VOC_ppb = df_data_VOC.loc[:,'formaldehyde':'n-dodecane'] \
            .div(df_NMOG_EF_fil52.loc[:,'NMOG(mg/kg-fuel)'], axis=0) \
            .multiply(df_NMOG_ppb_fil52.loc[:,'NMOG (ppb)'],axis=0)

#######
# Filter for common rows between exp info+VOCs (52) and IVOCs (3)
df_S523_test_ID = pd.merge(df_S52_test_ID, pd.DataFrame(S3a_test_ID), how='inner', on='test_ID')
# rows for which there is NMOG and VOC data but no IVOC data
df_S52_noIVOC_test_ID = pd.merge(df_S52_test_ID, df_S523_test_ID, how='left', indicator=True) \
                            .query("_merge == 'left_only'") \
                            .drop('_merge',1)
df_NMOG_EF_fil523 = df_NMOG_EF.merge(df_S523_test_ID, how='inner', on='test_ID')
df_NMOG_ppb_fil523 = df_NMOG_ppb.merge(df_S523_test_ID, how='inner', on='test_ID')
df_data_IVOC = df_S3_dup.merge(df_S523_test_ID, how='inner', on='test_ID')
# Divide by NMOG_EF and multiply by NMOG ppb
df_IVOC_ppb = df_data_IVOC.loc[:,'Dodecane':'Unspeciated cyclic compound Bin22'] \
                .div(df_NMOG_EF_fil523.loc[:,'NMOG(mg/kg-fuel)'], axis=0) \
                .multiply(df_NMOG_ppb_fil523.loc[:,'NMOG (ppb)'],axis=0)

#######
# To be continued for SVOCs


#df_52_noIVOC = df_52_noIVOC.dropna(axis=1, how='all')
#df_SVOC_volbasis = 



In [147]:
# Calculate missing values for IVOCs given an average composition
df_IVOC_perNMOGppb = df_data_IVOC.loc[:,'Dodecane':'Unspeciated cyclic compound Bin22'] \
                .div(df_NMOG_EF_fil523.loc[:,'NMOG(mg/kg-fuel)'], axis=0)
## Find the average composition
IVOCspecies_mean = pd.DataFrame(df_IVOC_perNMOGppb.mean(axis=0))
IVOCspecies_normalizeddist = IVOCspecies_mean.div(IVOCspecies_mean.sum(axis=0), axis=1)
IVOCspecies_normalizeddist.sum(axis=0) # Check, this should equal 1

# The NMOG for exps without IVOCs
df_NMOG_ppb.merge(df_S52_noIVOC_test_ID, how='inner', on='test_ID')

# What fraction of NMOG is in the VOCs? Let's assume that the IVOCs take up the rest if VOC fraction > 0.6, otherwise assume IVOCs take up 0.4
df_VOC_perNMOGppb = df_data_VOC.loc[:,'formaldehyde':'n-dodecane'] \
            .div(df_NMOG_EF_fil52.loc[:,'NMOG(mg/kg-fuel)'], axis=0)
#pd.merge(df_VOC_perNMOGppb, df_S52_noIVOC_test_ID, how='inner', on='test_ID')
# Attach the test IDs back
df_VOC_perNMOGppb_test_ID = pd.concat([df_S52_test_ID, df_VOC_perNMOGppb], axis=1)
# Use the test IDs to filter out exp without IVOC data
df_VOC_perNMOGppb_noIVOC = pd.merge(df_VOC_perNMOGppb_test_ID, df_S52_noIVOC_test_ID, how='inner', on='test_ID')
df_VOC_perNMOGppb_noIVOC.iloc[:,1:-1].sum(axis=1)
#with pd.option_context('display.max_rows', 999):
    #print(IVOCspecies_mean.div(IVOCspecies_mean.sum(axis=0), axis=1))

0     0.533426
1     0.385313
2     0.369487
3     0.607451
4     0.834257
5     0.599405
6     0.706197
7     0.747085
8     1.032738
9     0.977219
10    0.823568
11    0.397939
dtype: float64

In [148]:
# attach test IDs to NMOG_EF
df_NMOG_EF  = pd.concat([pd.DataFrame(S2_test_ID), pd.DataFrame(NMOG_EF)], axis=1)
df_NMOG_ppb = pd.concat([pd.DataFrame(S5_test_ID), pd.DataFrame(NMOG_ppb)], axis=1)
# get the VOCs into a volume basis
VOC_selection = df_S2.merge(how='inner',on='test_ID').merge()
VOC_selection = 
#df_VOC_ppb = 
#df_IVOC_volbasis = 
#df_SVOC_volbasis = 
# get the IVOCs that have EFs into a volume basis

# get the SVOCs that have EFs into a volume basis

SyntaxError: invalid syntax (<ipython-input-148-2537933c6f02>, line 6)

### now comes the hard part: parsing all the organics and separating them into categories that fit into SOM

### This (old) section attempts to group by category first, then by experiment

In [149]:
# I tried to use a groupby method to get species and carbon numbers together, 
# but groupby does not have the flexibility I need
# So instead, I'm going about it with a hack.
# Essentially, what I am doing is going through each block
# and making a list of the indices that correspond to the selected category
# Then, I'll index into the original dataframe and sum up the concentration
# of the species that match the category

# create a list of group categories to pass into the parsing function
categories = ['isoprene', 'aPinene', 'sesq', 'benzene', 'toluene', 'xylenes', 'naphthalene', 'n-alkane', 'b-alkane', 'c-alkane']
cat_files = (S2_cat, S3a_cat, S3b_cat, S4_cat)
data_files = (df_S2, df_S3a, df_S3b, df_S4)
# I want a list of reversed numbers to make transposing them easier later
carbon_num = [x+1 for x in list(reversed(range(23)))] # create a list from 23 to 1, to represent carbon numbers
# Create dataframes to store the results
S2_catd = pd.DataFrame(S2_test_ID)
S3a_catd = pd.DataFrame(S3a_test_ID)
S3b_catd = pd.DataFrame(S3b_test_ID)
S4_catd = pd.DataFrame(S4_test_ID)
cat_files_list = [S2_catd, S3a_catd, S3b_catd, S4_catd]

In [150]:
# Now we loop through all the files, through all the categories, and if the category is alkanes,
# we loop through the carbon numbers.
for i in range(len(data_files)):
    for j in range(len(categories)):
        # Check to see if you're doing alkanes. If you are, you have to loop through carbon numbers too
#         if  (  (categories[j] == 'n-alkane')
#              | (categories[j] == 'b-alkane')
#              | (categories[j] == 'c-alkane')
#             ):
        for k in range(len(carbon_num)):
            # This pulls out indices for species that match each alkane type and carbon number
            # The category matrix is missing the test_ID column though
            # , so all the indices are offset from the database columns by 1 
            temp_index_list = cat_files[i].index[ (cat_files[i]['Category']==categories[j]) 
                                                 & (cat_files[i]['Carbon number']==carbon_num[k]) ].tolist()
            # Now we find and sum up the species
            # Remember, we need the +1 in the data_files indices to account for the test_ID column
            data_index_list = [x+1 for x in temp_index_list]
            temp_conc = data_files[i].iloc[:,(data_index_list)].sum(axis=1)
            # Store it back into the dataframe
            cat_files_list[i][categories[j],carbon_num[k]] = temp_conc                
#         else:            
#             # This pulls out the indices for a category, in a file
#             temp_index_list = cat_files[i].index[cat_files[i]['Category'] == categories[j]].tolist()
#             # This should also require +1 to all the indices because of the test_ID column
#             data_index_list = [x+1 for x in temp_index_list]
#             # This pulls out, then sums, the concentrations matching the categories
#             temp_conc = data_files[i].iloc[:,data_index_list].sum(axis=1)
#             # Store it in a new dataframe for categorized data
#             # This is a list of arrays, where each item in the list corresponds to a different VOC (VOC, SVOC, IVOC)
#             # Each VOC is then categorized
#             cat_files_list[i][categories[j]] = temp_conc


In [151]:
# Now we need to merge them together, and remove excess columns
# Columns with all 0's are unnecessary.
# I'm taking some of the previous merging code and adapting it here

# Merge 3a and 3b together because they share all rows with each other
S3_catd = pd.concat([S3a_catd, S3b_catd], ignore_index=True
                   ).groupby(['test_ID'], as_index=False).sum()
# Taking just the test IDs from 5, we want to merge them with 2. I'll grab the SOA concentration later.
S52_catd = pd.merge(pd.DataFrame(S5_test_ID), S2_catd, how='inner', on='test_ID')

In [152]:
# Now, with 5 and 2 combined, we want to combine it with 3
S523_concat_temp = pd.concat([S52_catd, S3_catd]).groupby(['test_ID'], as_index=False).sum()
# Unfortunately, concat does not eliminate experiments not present in both
# This poses the problem that some experiments will be a combination of more than ibe
# So, we will have to add a selection process
# Select the test IDs from the merge with 5,2,3.
S523_test_ID = df_523.T.iloc[0].astype(int)
S523_catd = pd.merge(pd.DataFrame(S523_test_ID), S523_concat_temp, how='inner', on='test_ID')

In [153]:
# Now do the same thing with 4
S5234_concat_temp = pd.concat([S523_catd, S4_catd]).groupby(['test_ID'], as_index=False).sum()
S5234_test_ID = df_5234.T.iloc[0].astype(int)
S5234_catd = pd.merge(pd.DataFrame(S5234_test_ID), S5234_concat_temp, how='inner', on='test_ID')

#  Create SOM compatible format for a dataset



In [154]:
# Create a function that will take any dataframe of organics
# and output them in a SOM compatible format

In [155]:
# Great, it appears to work. 
# Now, we should take out the zero columns 
# I would like to keep the 0 columns in the alkanes
# , because it would be easier to copy/paste into Igor.
# S5234_catd_nonalkane = S5234_catd.iloc[:,0:7].loc[:,(S5234_catd.iloc[:,0:7] != 0).any(axis=0)]
# # Get the alkanes out
# S5234_catd_alkane    = S5234_catd.iloc[:,7:]
# # Now put the two together
# S5234_catd_nonzero = pd.concat([S5234_catd_nonalkane, S5234_catd_alkane], axis=1)

# Removed code that pared down the nonzeros. We're just going to leave them in.
S5234_catd_nonzero = S5234_catd



In [156]:
# Now we need to make this dataframe of Emission Factors (EF) and change them into ppb (or more likely, ppm!) for SOM
# First, let's make it into fractions
# Divide the EF for each category by the total EF

# Get the EF for these experiments, and attach them on
# First attach the test_ID labels to the NMOGs
df_NMOG_EF = pd.concat([pd.DataFrame(S2_test_ID), pd.DataFrame(NMOG_EF)], axis=1)
# Then filter them through the combined 5234 experiments to get only the test IDs that match them
S5234_catd_NMOG_EF = pd.merge(S5234_catd_nonzero, df_NMOG_EF, how='inner', on='test_ID')

# Now we divide all the EFs by the total NMOG EF to get the fraction
S5234_catd_frac_temp = S5234_catd_NMOG_EF.iloc[:,1:-1].div(S5234_catd_NMOG_EF.loc[:,'NMOG(mg/kg-fuel)'], axis=0)
# Add back the test ID column in the front, and NMOG_EF column in the back
S5234_catd_frac = pd.concat([  S5234_catd_NMOG_EF.iloc[:,0]
                             , S5234_catd_frac_temp
                             , S5234_catd_NMOG_EF.iloc[:,-1]
                            ], axis=1
                           )

In [157]:
# Now we have to make them into ppb (later, ppm)
# Similar procedure as above.
# Attach test ID labels 
df_NMOG_ppb = pd.concat([pd.DataFrame(S5_test_ID), pd.DataFrame(NMOG_ppb)], axis=1)
# Filter to get only the fractions that match the filtered 5234 list
S5234_frac_NMOG_ppb = pd.merge(S5234_catd_frac, df_NMOG_ppb, how='inner', on='test_ID')

# Now we multiply all the categories by the EF ppb to get the ppb for each category
S5234_catd_ppb_temp = S5234_frac_NMOG_ppb.iloc[:,1:-2].mul(S5234_frac_NMOG_ppb.loc[:,'NMOG (ppb)'], axis=0)
S5234_catd_ppm_temp = S5234_catd_ppb_temp.div(1000)

In [158]:
# This is the final form of the data for now.
# Actually putting it into SOM form will require some row/column manipulation
# , but we will not do that here. It would take more work than necessary

# Two forms are presented here, one in ppb, one in ppm.
# Although SOM asks for a user input in ppb (a more common unit)
# , the calculations in SOM uses ppm
# , and SOM takes ppb from user input and changes it to ppm.
S5234_catd_ppb = pd.concat([ S5234_frac_NMOG_ppb.loc[:,'test_ID']
                            ,S5234_catd_ppb_temp
                            ,S5234_frac_NMOG_ppb.loc[:,'NMOG(mg/kg-fuel)']
                            ,S5234_frac_NMOG_ppb.loc[:,'NMOG (ppb)']
                           ], axis=1
                          )
S5234_catd_ppm = pd.concat([ S5234_frac_NMOG_ppb.loc[:,'test_ID']
                            ,S5234_catd_ppm_temp
                            ,S5234_frac_NMOG_ppb.loc[:,'NMOG(mg/kg-fuel)']
                            ,S5234_frac_NMOG_ppb.loc[:,'NMOG (ppb)']
                           ], axis=1
                          )

In [159]:
# See which experiments have issues with mass closure
# Mass closure is where the sum of all compounds is higher than the detected total
# Zhao 2015, EST suggests that mass closure could be due to 
    # background IVOCs in the CVS, "which corresponds up to 42% of measured IVOCs in during the UDDS"
    # , or, only a portion of the IVOCs are measured as NMHCs (non-methane hydrocarbons)
S5234_catd_ppb.to_csv('S5234_catd_ppb.csv')
#S5234_catd_ppb.iloc[:,1:-2].sum(axis=1)

In [160]:
# Oh wait, Chris made SOM take in ppm instead of ppb. 
# so, here's the ppm file:
S5234_catd_ppm.to_csv('S5234_catd_ppm.csv')

In [161]:
# Now I want to reorganize the data into separate experiments
# And each experiment should be reformatted for SOM input

# First, I need the column labels
label_names = list(S5234_catd_ppm)

# for each experiment (row),
    # Look at the organics that require a list.
#alkane_list = ['n-alkane', 'b-alkane', 'c-alkane']

# This is the index for non-alkanes. Initialize to length of data, decrease if alkanes exists
min_index = S5234_catd_ppm.shape[1]

# For each row in the data: 
for k in range(S5234_catd_ppm.shape[0]):
    exp_row = pd.DataFrame(S5234_catd_ppm.iloc[k])

    # Initialize an array to store everything
    df_SOM_ready = pd.DataFrame()
    # Loop through the alkanes
    for j in range(len(categories)):
        # Get the indices for the alkanes
        indices = [i for i, elem in enumerate(label_names) if categories[j] in elem] 
        # Get the species concentrations (in a column)
        species = exp_row.iloc[indices].reset_index(drop=True)
        species = species.rename(index=str, columns={k:categories[j]})
        # Concat this together with the entire array
        df_SOM_ready = pd.concat([df_SOM_ready, species], axis=1, sort=False)
        # change minimum to the smallest species
        if min(indices) < min_index:
            min_index = min(indices)

    # Get the remaining list of organics
#     df_non_alkanes = exp_row[1:min_index].T.reset_index(drop=True)
    df_SOM_ready = df_SOM_ready.reset_index(drop=True)

    # Combine them together into one dataframe    
#     df_SOM_ready = pd.concat([df_non_alkanes, df_SOM_ready], axis=1, sort=False)

    # Replace all the NaN with 0s
    df_SOM_ready = df_SOM_ready.fillna(0)
    
    # Give a name to the final dataframe
    df_SOM_ready.name = str(exp_row.loc['test_ID'].iloc[0].astype(int))
    # Save it into a file, using its name
    df_SOM_ready.to_csv(df_SOM_ready.name + '_ppm' +'.csv')

In [162]:
filtered = df_S5[df_S5['test_ID'].isin(S5234_catd_ppm['test_ID'])]
# I want the details from S5 for the IDs that match the group with complete data
filtered.to_csv('S5_complete_data.csv')

# End

# Missing SVOCs

In [ ]:
# Discovered that even those experiments where we have speciated NMOG, SVOC, IVOC data, 
# we might not have NMOG, OH exposure data! Why are they missing?
# Case in point, test_ID 1032443. We have NMOG, IVOC, SVOC data, but no data in table 5.
# Hmm, looks like they sampled, but did not run chamber aging experiments. So there should be
# some POA data out there from which the ratios were constructed?

In [ ]:
# Ok, now to replace the info for S523_no4, where we need to infer the SVOC concentrations.
# Look at the ones with information, then extract the *median* ratio of SVOCs to POA. 
# How many data points should I have?
# There are 42 data points in the spreadsheet for SVOCs. Let's try to get some information from that.

In [158]:
# First, let's merge the sheets that have data on POA and on SVOCs
# Then, I need to process the volume concentrations by dividing by the NMOG EF
# And then multiplying by the NMOG ppb
# Lastly, I need to mulitply by the MW (and other factors) to get the species ug/m3
# This can then be compared to the POA (ug/m3) to get a ratio
raw_54 = pd.merge(df_S5, df_S4, how='inner', on='test_ID') # This shows what is common between the two 

In [159]:
raw54NMOG = pd.merge(raw_54, df_NMOG_EF, how='inner', on='test_ID')

In [160]:
SVOC_ind1 = raw54NMOG.columns.get_loc('SVOC B23')
SVOC_ind2 = raw54NMOG.columns.get_loc('SVOC B32')
massfrac54 = raw54NMOG.iloc[:,SVOC_ind1:SVOC_ind2+1].div(raw54NMOG.loc[:,'NMOG(mg/kg-fuel)'], axis=0)#.mul(raw54NMOG.loc[:,'NMOG (ppb)'], axis=0)

In [170]:
raw54NMOG

,test_ID,Vehicle,NMOG (ppb),Propene (ppm),d-butanol (ppm),NO (ppb),OH exposure (molecules cm-3 hr),OA (ug m-3),SOA (ug/m3),SVOC B23,SVOC B24,SVOC B25,SVOC B26,SVOC B27,SVOC B28,SVOC B29,SVOC B30,SVOC B31,SVOC B32,NMOG(mg/kg-fuel)
0,1027971,ULEV-13,140.000000,0.060,0.06,41.125064,1.051860e+07,15.45310,14.653114,0.222087,0.150233,0.123054,0.077324,0.097566,0.063780,0.042960,0.038761,0.000000,0.000000,404.804021
1,1032303,PreLEV-10,2479.452874,1.000,0.06,1503.153211,7.681165e+06,37.81970,15.119657,1.224966,1.298569,1.637930,1.771539,2.004760,1.857519,1.550107,1.176829,0.974124,5.070090,5615.329370
2,1032304,LEV-4,316.066094,0.200,0.12,206.965036,8.870178e+06,6.97066,6.470656,0.129602,0.140586,0.110395,0.106966,0.197531,0.118296,0.079112,0.047967,0.004547,0.023351,1490.044697
3,1032309,ULEV-16,196.166703,0.133,0.06,97.399509,1.171945e+07,2.92793,2.327932,0.277302,0.361576,0.350843,0.251041,0.285395,0.145590,0.081400,0.047279,0.005475,0.020667,587.680567
4,1032321,ULEV-16,287.377911,0.133,0.06,121.574776,1.764747e+07,4.36275,3.762747,0.100531,0.115886,0.107955,0.076079,0.088038,0.060349,0.047256,0.029525,0.010751,0.039184,743.132218
5,1032342,ULEV-14,143.336802,0.600,0.06,302.694690,1.025585e+07,8.59225,7.892251,0.513019,0.579543,0.509283,0.322277,0.339198,0.195191,0.125857,0.081285,0.037083,0.198881,410.020715
6,1032360,ULEV-11,58.829398,0.200,0.06,145.948455,8.304225e+06,1.22520,0.625198,0.183367,0.283075,0.328502,0.198214,0.256643,0.102923,0.071719,0.045386,0.021155,0.111018,77.711511
7,1032393,LEV-9,191.991042,0.530,0.06,401.382222,7.969561e+06,6.12593,5.325932,0.168784,0.215621,0.224182,0.189076,0.143211,0.089717,0.066473,0.034717,0.019910,0.065624,951.329132
8,1032440,PreLEV-9,10641.436226,0.000,0.06,2502.762618,5.542464e+06,19.84880,10.648837,1.484359,1.517917,1.429203,0.660047,0.815746,0.425883,0.374265,0.206099,0.111892,0.837704,34983.464067
9,1032442,PreLEV-4,1401.118018,0.130,0.06,671.372952,6.772551e+06,5.05499,3.054992,3.797433,27.460881,9.150117,25.383611,9.643177,5.296952,5.049557,3.309277,0.691184,4.241500,8824.643651


In [172]:
ySVOC = pd.DataFrame(raw54NMOG.loc[:,'SVOC B23':'SVOC B32'].sum(axis=1)).rename(index=str, columns={0:'SVOC'})
XSVOC = pd.DataFrame(raw54NMOG.loc[:,'OA (ug m-3)'])
regrSVOC = linear_model.LinearRegression(fit_intercept=False)
regrSVOC.fit(XSVOC,ySVOC)
print('R2 = ' + str(regrSVOC.score(XSVOC,ySVOC)))
print('slope = ' + str(regrSVOC.coef_))
print('intercept = ' + str(regrSVOC.intercept_))

R2 = -0.126302531855
slope = [[ 0.68283989]]
intercept = 0.0


In [31]:
# This is the mass fraction of SVOCs to NMOG.
# This does not match Yunliang's fit.
massfrac54

,SVOC B23,SVOC B24,SVOC B25,SVOC B26,SVOC B27,SVOC B28,SVOC B29,SVOC B30,SVOC B31,SVOC B32
0,0.000549,0.000371,0.000304,0.000191,0.000241,0.000158,0.000106,0.000096,0.000000,0.000000
1,0.000218,0.000231,0.000292,0.000315,0.000357,0.000331,0.000276,0.000210,0.000173,0.000903
2,0.000087,0.000094,0.000074,0.000072,0.000133,0.000079,0.000053,0.000032,0.000003,0.000016
3,0.000472,0.000615,0.000597,0.000427,0.000486,0.000248,0.000139,0.000080,0.000009,0.000035
4,0.000135,0.000156,0.000145,0.000102,0.000118,0.000081,0.000064,0.000040,0.000014,0.000053
5,0.001251,0.001413,0.001242,0.000786,0.000827,0.000476,0.000307,0.000198,0.000090,0.000485
6,0.002360,0.003643,0.004227,0.002551,0.003303,0.001324,0.000923,0.000584,0.000272,0.001429
7,0.000177,0.000227,0.000236,0.000199,0.000151,0.000094,0.000070,0.000036,0.000021,0.000069
8,0.000042,0.000043,0.000041,0.000019,0.000023,0.000012,0.000011,0.000006,0.000003,0.000024
9,0.000430,0.003112,0.001037,0.002876,0.001093,0.000600,0.000572,0.000375,0.000078,0.000481


In [32]:
# This is the volume fraction of SVOCs to NMOG.
# From what I can tell, these numbers are way too low
# Compared to what Yunliang reported, or .85 for pre-LEV, 1.25 for LEV, and 1.19 for ULEV)
# So it's definitely not volume fraction.
volfrac54 = massfrac54.div(S4_MW.iloc[0,:], axis='columns') * avgMW[0] 

In [33]:
# Transform volume fractions to ppb, then to ug/m3
# These numbers are way too high for some, and way too low for others
volfrac54.mul(raw54NMOG.loc[:,'NMOG (ppb)'], axis=0).mul(S4_MW.iloc[0,:]) * .0409 

,SVOC B23,SVOC B24,SVOC B25,SVOC B26,SVOC B27,SVOC B28,SVOC B29,SVOC B30,SVOC B31,SVOC B32
0,0.201050,0.136002,0.111398,0.070000,0.088324,0.057739,0.038891,0.035089,0.000000,0.000000
1,1.415798,1.500868,1.893097,2.047521,2.317074,2.146895,1.791593,1.360163,1.125879,5.859940
2,0.071960,0.078058,0.061295,0.059391,0.109676,0.065682,0.043926,0.026633,0.002525,0.012965
3,0.242289,0.315922,0.306545,0.219344,0.249361,0.127208,0.071122,0.041309,0.004784,0.018058
4,0.101762,0.117305,0.109277,0.077010,0.089115,0.061087,0.047835,0.029886,0.010883,0.039663
5,0.469442,0.530316,0.466024,0.294902,0.310386,0.178611,0.115167,0.074380,0.033933,0.181988
6,0.363351,0.560929,0.650945,0.392772,0.508553,0.203947,0.142115,0.089934,0.041919,0.219988
7,0.089162,0.113904,0.118426,0.099881,0.075652,0.047394,0.035115,0.018340,0.010517,0.034667
8,1.181880,1.208600,1.137963,0.525544,0.649515,0.339098,0.297998,0.164100,0.089091,0.666999
9,1.578209,11.412712,3.802779,10.549401,4.007694,2.201407,2.098590,1.375332,0.287255,1.762763


In [35]:
ULEV54 = raw_54[raw_54['Vehicle'].str.contains('ULEV')]

ULEVnames = list(df_S4)
ULEVnames.remove('test_ID')
ULEV_SVOC = ULEV54[ULEVnames].sum(axis=1)

ULEV_POA = ULEV54['OA (ug m-3)'] - ULEV54['SOA (ug/m3)']

(ULEV_SVOC/ULEV_POA).median()

2.6699940701416063

In [36]:
(ULEV_SVOC/(ULEV54['OA (ug m-3)'])).median()

0.33770178129964873

In [37]:
PreLEV54 = raw_54[raw_54['Vehicle'].str.contains('PreLEV')]
PreLEVnames = list(df_S4)
PreLEVnames.remove('test_ID')
PreLEV_SVOC = PreLEV54[PreLEVnames].sum(axis=1)
PreLEV_POA = PreLEV54['OA (ug m-3)']
(PreLEV_SVOC/PreLEV_POA).median()

1.1141144326774899

# End SVOCs

# Let's look at IVOCs instead 

In [ ]:
# SVOC doesn't quite work. How about IVOC?
# IVOCs are dependent on NMOG, which we have more data on. 
# We also have to separate that from the single ring aromatic compounds,
# which have a different ratio

In [106]:
# Merge NMOG and the IVOCs together
# df_S3 here has already been transformed into a volume fraction in cell 11.
S3_NMOG = pd.merge(df_S3, df_NMOG_EF, how='inner', on='test_ID')
# We can extract the column names of the IVOCs using regex or filter. List comprehension also works
#ULEV_IVOCnames = raw_53.columns[raw_53.columns.str.contains(pat = 'Unspeciated')]
#ULEV_IVOCnames =  raw_53.filter(regex='Unspeciated').columns

S3_MW = pd.concat([S3a_MW, S3b_MW], axis=1)
# Everything has already been transformed into a volume fraction
volfrac3 = S3_NMOG.iloc[:,1:-1].div(S3_NMOG.loc[:,'NMOG(mg/kg-fuel)'], axis=0) 

In [156]:
## Test linear regression model here
X2 = pd.DataFrame(S3_NMOG.loc[:,'NMOG(mg/kg-fuel)'])
y2 = pd.DataFrame(S3_NMOG.iloc[:,1:-1].sum(axis=1))
regr2 = linear_model.LinearRegression(fit_intercept=False)
regr2.fit(X2,y2)
print('R2 = ' + str(regr2.score(X2,y2)))
print('Slope = ' + str(regr2.coef_))
print('Intercept = ' + str(regr2.intercept_))

R2 = 0.920039160287
Slope = [[ 0.03820881]]
Intercept = 0.0


In [277]:
volfrac3

,Dodecane,Tridecane,Tetradecane,Pentadecane,Hexadecane,Heptadecane,Octadecane,Nonadecane,Eicosane,Heneicosane,Docosane,"2,6,10-Trimethylundecane","2,6,10-Trimethyldodecane","2,6,10-Trimethyltridecane","2,6,10-Trimethylpentadecane",Pristane,Phytane,Hexylcyclohexane,Heptylcyclohexane,Octylcyclohexane,Nonylcyclohexane,Decylcyclohexane,Undecylcyclohexane,Dodecylcyclohexane,Tridecylcyclohexane,Tetradecylcyclohexane,Pentadecylcyclohexane,Hexadecylcyclohexane,Heptadecylcyclohexane,Naphthalene,2-methylnaphthalene,1-methylnaphthalene,C2-naphthalene,C3-naphthalene,C4-naphthalene,Acenaphthylene,Acenaphthene,Fluorene,C1-Fluorene,Phenanthrene,Anthracene,C1-Phenanthrene/anthracene,C2-Phenanthrene/anthracene,Fluoranthene,Pyrene,C1-Fluoranthene/pyrene,Pentylbenzene,Hexylbenzene,Heptylbenzene,Octylbenzene,Nonylbenzene,Decylbenzene,Undecylbenzene,Dodecylbenzene,Tridecylbenzene,Tetradecylbenzene,Pentadecylbenzene,Unspeciated b-alkane bin12,Unspeciated b-alkane bin13,Unspeciated b-alkane bin14,Unspeciated b-alkane bin15,Unspeciated b-alkane bin16,Unspeciated b-alkane bin17,Unspeciated b-alkane bin18,Unspeciated b-alkane bin19,Unspeciated b-alkane bin20,Unspeciated b-alkane bin21,Unspeciated b-alkane bin22,Unspeciated cyclic compound Bin12,Unspeciated cyclic compound Bin13,Unspeciated cyclic compound Bin14,Unspeciated cyclic compound Bin15,Unspeciated cyclic compound Bin16,Unspeciated cyclic compound Bin17,Unspeciated cyclic compound Bin18,Unspeciated cyclic compound Bin19,Unspeciated cyclic compound Bin20,Unspeciated cyclic compound Bin21,Unspeciated cyclic compound Bin22
0,0.000155,0.000090,0.000053,0.000019,0.000015,0.000007,0.000006,0.000003,0.000004,0.000007,0.000022,0.000052,0.000000,0.000000,0.000003,0.000006,0.000006,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000003,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.001390,0.000429,0.000238,0.000289,0.000096,0.000000,0.000025,0.000017,0.000011,0.000000,0.000030,0.000000,0.000012,0.000000,0.000004,0.000004,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.002112,0.000782,0.000513,0.000288,0.000198,0.000148,0.000095,0.000071,0.000059,0.000041,0.000036,0.009673,0.003522,0.002023,0.001260,0.000705,0.000418,0.000326,0.000207,0.000165,0.000109,0.000013
1,0.000285,0.000153,0.000189,0.000097,0.000096,0.000138,0.000046,0.000034,0.000048,0.000080,0.000206,0.000000,0.000000,0.000000,0.000000,0.000000,0.000034,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.003001,0.000422,0.000410,0.000186,0.000000,0.000000,0.000052,0.000000,0.000000,0.000000,0.000262,0.000000,0.000000,0.000000,0.000053,0.000067,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.006708,0.003621,0.002036,0.001483,0.001202,0.000982,0.000627,0.000502,0.000424,0.000323,0.000287,0.002884,0.000000,0.000465,0.000570,0.000000,0.000121,0.000032,0.000680,0.000654,0.000141,0.000000
2,0.000129,0.000083,0.000044,0.000026,0.000018,0.000010,0.000005,0.000003,0.000004,0.000005,0.000012,0.000089,0.000000,0.000000,0.000000,0.000006,0.000004,0.000019,0.000007,0.000006,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.001043,0.000313,0.000164,0.000304,0.000103,0.000000,0.000036,0.000011,0.000002,0.000000,0.000027,0.000000,0.000009,0.000000,0.000003,0.000004,0.000000,0.000041,0.000015,0.000004,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.001922,0.000704,0.000642,0.000314,0.000197,0.000139,0.000092,0.000071,0.000060,0.000045,0.000037,0.008042,0.002597,0.001576,0.000917,0.000494,0.000345,0.000296,0.000188,0.000120,0.000097,0.000000
3,0.000290,0.000200,0.000144,0.000073,0.000066,0.000067,0.000051,0.000056,0.000071,0.000144,0.000201,0

In [278]:
# Look at the data
IVOCmean = pd.DataFrame(volfrac3.mean())
IVOCmean = IVOCmean.reset_index()
IVOCmean.columns = ['species', 'mean']

IVOCmedian = pd.DataFrame(volfrac3.median())
IVOCmedian = IVOCmedian.reset_index()
IVOCmedian.columns = ['species', 'median']

IVOCstd = pd.DataFrame(volfrac3.std())
IVOCstd = IVOCstd.reset_index()
IVOCstd.columns = ['species','std']

IVOCstats = pd.merge(IVOCmean, IVOCmedian, on='species')
IVOCstats = pd.merge(IVOCstats, IVOCstd, on='species')
#IVOCstats
# Can we assume that IVOCstats is a "typical profile" for emissions?
# Do we need to separate by vehicle emissions standards?
# I still have doubts about extended these averages to the fleet
# Because there are major differences between experimental runs
# But, in the interest of trying something new, 
# I am running with the mean, and seeing if the results say anything


In [279]:
# This is the mean fraction of each IVOC species w.r.t. NMOG.
# Get the concentration by multiplying by the NMOG (ppb)
volmean3 = volfrac3.mean()
## Then, normalize it as a fraction of the IVOC/NMOG ratio, ratioNMOG_IVOC
# volmean3sum = volfrac3.mean().sum()
## This is the fraction of IVOC:NMOG that is attributed to each species
## To get concentration, multiply this by the IVOC:NMOG ratio, then by NMOG (ppb)
# IVOCvolmeanNormalized = volmean3 / volmean3sum
df_volmean3 = pd.DataFrame(volmean3)#.reset_index()
#df_volmean3.columns = ['species','fraction']
df_volmean3.columns = ['fraction']
df_volmean_IVOC = df_volmean3.transpose()

In [280]:
df_volmean_IVOC

,Dodecane,Tridecane,Tetradecane,Pentadecane,Hexadecane,Heptadecane,Octadecane,Nonadecane,Eicosane,Heneicosane,Docosane,"2,6,10-Trimethylundecane","2,6,10-Trimethyldodecane","2,6,10-Trimethyltridecane","2,6,10-Trimethylpentadecane",Pristane,Phytane,Hexylcyclohexane,Heptylcyclohexane,Octylcyclohexane,Nonylcyclohexane,Decylcyclohexane,Undecylcyclohexane,Dodecylcyclohexane,Tridecylcyclohexane,Tetradecylcyclohexane,Pentadecylcyclohexane,Hexadecylcyclohexane,Heptadecylcyclohexane,Naphthalene,2-methylnaphthalene,1-methylnaphthalene,C2-naphthalene,C3-naphthalene,C4-naphthalene,Acenaphthylene,Acenaphthene,Fluorene,C1-Fluorene,Phenanthrene,Anthracene,C1-Phenanthrene/anthracene,C2-Phenanthrene/anthracene,Fluoranthene,Pyrene,C1-Fluoranthene/pyrene,Pentylbenzene,Hexylbenzene,Heptylbenzene,Octylbenzene,Nonylbenzene,Decylbenzene,Undecylbenzene,Dodecylbenzene,Tridecylbenzene,Tetradecylbenzene,Pentadecylbenzene,Unspeciated b-alkane bin12,Unspeciated b-alkane bin13,Unspeciated b-alkane bin14,Unspeciated b-alkane bin15,Unspeciated b-alkane bin16,Unspeciated b-alkane bin17,Unspeciated b-alkane bin18,Unspeciated b-alkane bin19,Unspeciated b-alkane bin20,Unspeciated b-alkane bin21,Unspeciated b-alkane bin22,Unspeciated cyclic compound Bin12,Unspeciated cyclic compound Bin13,Unspeciated cyclic compound Bin14,Unspeciated cyclic compound Bin15,Unspeciated cyclic compound Bin16,Unspeciated cyclic compound Bin17,Unspeciated cyclic compound Bin18,Unspeciated cyclic compound Bin19,Unspeciated cyclic compound Bin20,Unspeciated cyclic compound Bin21,Unspeciated cyclic compound Bin22
fraction,0.000311,0.000185,0.000089,0.000047,0.000044,0.000036,0.000035,0.000019,0.000037,0.000054,0.00011,0.000032,0.000009,0.000002,0.000009,0.000023,0.000018,0.00002,0.000007,0.000003,0.000002,0.000003,0.000002,0.000001,3.444367e-07,0.000001,6.921717e-07,6.638630e-08,6.654722e-08,0.003194,0.000938,0.000497,0.000418,0.000126,0.000008,0.000072,0.000018,0.000026,0.000011,0.000099,0.000013,0.000022,0.000006,0.000019,0.000023,0.000002,0.000042,0.000017,0.000003,7.194019e-07,4.888773e-07,1.588083e-07,7.769128e-08,4.194494e-08,2.433677e-08,1.689501e-08,1.612157e-08,0.004249,0.001985,0.000988,0.000691,0.0005,0.000396,0.000327,0.000277,0.000312,0.000224,0.000272,0.012584,0.005198,0.002394,0.001142,0.001419,0.000812,0.000937,0.000782,0.00098,0.000725,0.000849


In [281]:
# filter out the scenarios without IVOCs
df_trial = df_S2.merge(df_S3.drop_duplicates(),on='test_ID',how='left',indicator=True)
df_IVOC_missing = df_trial[df_trial['_merge'] =='left_only']
df_IVOC_missing = df_IVOC_missing.dropna(axis=1).drop(columns='_merge')

# Get the vol fractions
NMOG_IVOC_missing = pd.DataFrame(df_IVOC_missing.pop('NMOG(mg/kg-fuel)'))
volfrac = df_IVOC_missing.iloc[:,1:-1].div(NMOG_IVOC_missing.loc[:,'NMOG(mg/kg-fuel)'], axis=0)

In [284]:
# Concentrations of all the missing IVOCs
missingIVOCs = pd.DataFrame(df_volmean_IVOC.values*NMOG_IVOC_missing.values, columns=df_volmean_IVOC.columns, index=NMOG_IVOC_missing.index)

In [285]:
# Combine with the VOCs
df_IVOC_filled = df_IVOC_missing.join(missingIVOCs, how='outer')

In [286]:
df_IVOC_filled

,test_ID,formaldehyde,acetaldehyde,acrolein,acetone,propanal,butanal,methyl-ethyl-ketone,methacrolein,benzaldehyde,crotonaldehyde,pentanal,m-tolualdehyde,hexanal,ethane,ethene,propane,propene,methyl-propane,ethyne,n-butane,"1,2-propadiene",trans-2-butene,1-butene,2-methyl-2-butene,cis-2-butene,"2,2-dimethyl-propane",2-methyl-butane,1-propyne,"1,2-butadiene","1,3-butadiene",trans-2-pentene,2-methyl-propene,1-pentene,2-methyl-1-butene,cis-2-pentene,1-buten-3-yne,2-butyne,1-butyne,"1,3-butadiyne",3-methyl-1-butene,n-pentane,"2-methyl-1,3-butadiene","3,3-dimethyl-1-butene","trans-1,3-pentadiene","2,2-dimethyl-butane",cyclo-pentene,4-methyl-1-pentene,3-methyl-1-pentene,cyclo-pentane,"2,3-dimethyl-butane","2,3-dimethyl-1-butene",methyl-tert-butyl-ether,4-methyl-cis-2-pentene,2-methyl-pentane,4-methyl-trans-2-pentene,3-methyl-pentane,1-hexene,2-methyl-1-pentene,n-hexane,trans-3-hexene,cis-3-hexene,trans-2-hexene,2-methyl-2-pentene,3-methyl-trans-2-pentene,3-methyl-cyclo-pentene,cis-2-hexene,3-methyl-cis-2-pentene,1-ethyl-tert-butyl-ether,"2,2-dimethyl-pentane",methyl-cyclo-pentane,"2,4-dimethyl-pentane","2,2,3-trimethyl-butane","3,4-dimethyl-1-pentene","2,4-dimethyl-1-pentene",1-methyl-cyclo-pentene,benzene,3-methyl-1-hexene,"3,3-dimethyl-pentane","2,4-dimethyl-2-pentene",cyclo-hexane,2-methyl-trans-3-hexene,4-methyl-trans-2-hexene,2-methyl-hexane,"2,3-dimethyl-pentane",cyclo-hexene,3-methyl-hexane,"trans-1,3-dimethyl-cyclo-pentane","cis-1,3-dimethyl-cyclo-pentane",3-ethyl-pentane,"trans-1,2-dimethyl-cyclo-pentane","2,2,4-trimethyl-pentane",1-heptene,3-methyl-trans-3-hexene,trans-3-heptene,n-heptane,"2,4,4-trimethyl-1-pentene",2-methyl-2-hexene,trans-2-heptene,3-ethyl-2-pentene,3-methyl-cis-2-hexene,"2,3-dimethyl-2-pentene",cis-2-heptene,methyl-cyclo-hexane,"2,2-dimethyl-hexane","2,4,4-trimethyl-2-pentene",ethyl-cyclo-pentane,"2,5-dimethyl-hexane","2,4-dimethyl-hexane","1,2,4-trimethyl-cyclo-pentane","3,3-dimethyl-hexane","(1a,2a,3b)-1,2,3-trimethyl-cyclo-pentane","2,3,4-trimethyl-pentane",toluene,"2,3,3-trimethyl-pentane","2,3-dimethyl-hexane",2-methyl-heptane,4-methyl-heptane,"3,4-dimethyl-hexane",3-methyl-heptane,"cis-1,2-dimethyl-cyclo-hexane","trans-1,4-dimethyl-cyclo-hexane","2,2,5-trimethyl-hexane",trans-1-methyl-3-ethyl-cyclo-pentane,cis-1-methyl-3-ethyl-cyclo-pentane,1-octene,"2,2,4-trimethyl-hexane",trans-4-octene,n-octane,trans-2-octene,"trans-1,3-dimethyl-cyclo-hexane","2,4,4-trimethyl-hexane",cis-2-octene,"2,3,5-trimethyl-hexane","2,4-dimethyl-heptane","cis-1,3-dimethyl-cyclohexane","2,6-dimethyl-heptane",ethyl-cyclo-hexane,"3,5-dimethyl-heptane",ethyl-benzene,"1,3,5-trimethyl-cyclo-hexane","2,3-dimethyl-heptane",m-xylene,p-xylene,4-methyl-octane,2-methyl-octane,3-methyl-octane,styrene,o-xylene,"2,2,4-trimethyl-heptane",1-methyl-4-ethyl-cyclo-hexane,"2,2,5-trimethyl-heptane",1-nonene,n-nonane,"3,3-dimethyl-octane",(1-methyl-ethyl) benzene,"2,3-dimethyl-octane","2,2-dimethyl-octane","2,5-dimethyl-octane","2,4-dimethyl-octane","2,6-dimethyl-octane",n-propyl-benzene,1-methyl-3-ethyl-benzene,1-methyl-4-ethyl-benzene,"1,3,5-trimethyl-benzene",2-methyl-nonane,1-methyl-2-ethyl-benzene,"1,2,4-trimethyl-benzene",(2-methyl-propyl) benzene,(1-methyl-propyl) benzene,n-decane,1-methyl-3-(1-methyl-ethyl) benzene,"1,2,3-trimethyl-benzene",1-methyl-4-(1-methyl-ethyl) benzene,"2,3-dihydro-indene",1-methyl-2-(1-methyl-ethyl) benzene,"1,3-diethyl-benzene",1-methyl-3-n-propyl-benzene,"1,4-diethyl-benzene",1-methyl-4-n-propyl-benzene,"1,3-dimethyl-5-ethyl-benzene","1,2-diethyl-benzene",1-methyl-2-n-propyl-benzene,"1,4-dimethyl-2-ethyl-benzene","1,3-dimethyl-4-ethyl-benzene","1,2-dimethyl-4-ethyl-benzene","1,3-dimethyl-2-ethyl-benzene",n-undecane,"1,2-dimethyl-3-ethyl-benzene","1,2,4,5-tetra-methyl-benzene","1,2,3,5-tetra-methyl-benzene","1-(1,1-dimethyl-ethyl)-2-methyl-benzene",5-methyl-indan,4-methyl-indan,1-ethyl-2-n-propyl-benzene,2-methyl-indan,"1,2,3,4-tetra-methyl-benzene",n-pentyl-benzene,1-methyl-2-n-butyl-benzene,naphthalene,"1-(1,1

In [189]:
# create a list of group categories to pass into the parsing function
categories = ['isoprene', 'aPinene', 'sesq', 'benzene', 'toluene', 'xylenes', 'naphthalene', 'n-alkane', 'b-alkane', 'c-alkane']
cat_files_IVOC = pd.concat([S2_cat, S3a_cat, S3b_cat])
data_files_IVOC = df_IVOC_filled
# I want a list of reversed numbers to make tranposing them easier later
carbon_num = [x+1 for x in list(reversed(range(23)))] # create a list from 23 to 1, to represent carbon numbers
# Create a list with a dataframe inside it 
testlist_IVOC = [pd.DataFrame(df_IVOC_filled.T.iloc[0].astype(int))]

# Now we loop through all the categories, and if the category is alkanes,
# we loop through the carbon numbers.

for j in range(len(categories)):
    # Check to see if you're doing alkanes. If you are, you have to loop through carbon numbers too
    if  (  (categories[j] == 'n-alkane')
         | (categories[j] == 'b-alkane')
         | (categories[j] == 'c-alkane')
        ):
        for k in range(len(carbon_num)):
            # This pulls out indices for species that match each alkane type and carbon number
            # The category matrix is missing the test_ID column though
            # , so all the indices are offset from the database columns by 1 
            temp_index_list = cat_files_IVOC.index[ (cat_files_IVOC['Category']==categories[j]) 
                                                 & (cat_files_IVOC['Carbon number']==carbon_num[k]) ].tolist()
            # Now we find and sum up the species
            # Remember, we need the +1 in the data_files indices to account for the test_ID column
            data_index_list = [x+1 for x in temp_index_list]
            temp_conc = data_files_IVOC.iloc[:,(data_index_list)].sum(axis=1)
            # Store it back into the dataframe
            testlist_IVOC[0][categories[j],carbon_num[k]] = temp_conc                
    else:            
        # This pulls out the indices for a category, in a file
        temp_index_list = cat_files_IVOC.index[cat_files_IVOC['Category'] == categories[j]].tolist()
        # This should also require +1 to all the indices because of the test_ID column
        data_index_list = [x+1 for x in temp_index_list]
        # This pulls out, then sums, the concentrations matching the categories
        temp_conc = data_files_IVOC.iloc[:,data_index_list].sum(axis=1)
        # Store it in a new dataframe for categorized data
        testlist_IVOC[0][categories[j]] = temp_conc
            

In [190]:
df_IVOC = testlist_IVOC[0].reset_index(drop=True)
df_IVOC.reset_index(drop=True)
label_names_IVOC = list(df_IVOC)
alkane_list = ['n-alkane', 'b-alkane', 'c-alkane']
min_index_IVOC = df_IVOC.shape[1]

In [194]:
for k in range(df_IVOC.shape[0]):
    exp_row_IVOC = pd.DataFrame(df_IVOC.iloc[k])

    # Initialize an array to store everything
    df_SOM_ready_IVOC = pd.DataFrame()
    # Loop through the alkanes
    for j in range(len(alkane_list)):
        # Get the indices for the alkanes
        indices_IVOC = [i for i, elem in enumerate(label_names_IVOC) if alkane_list[j] in elem] 
        # Get the alkane concentrations themselves (in a column)
        alkanes_IVOC = exp_row_IVOC.iloc[indices_IVOC].reset_index(drop=True)
        alkanes_IVOC = alkanes_IVOC.rename(index=str, columns={k:alkane_list[j]})
        # Concat this together with the entire array
        df_SOM_ready_IVOC = pd.concat([df_SOM_ready_IVOC, alkanes_IVOC], axis=1, sort=False)
        # change minimum to the smallest alkane
        if min(indices_IVOC) < min_index_IVOC:
            min_index_IVOC = min(indices_IVOC)

    # Get the remaining list of organics
    df_non_alkanes_IVOC = exp_row_IVOC[1:min_index_IVOC].T.reset_index(drop=True)
    df_SOM_ready_IVOC = df_SOM_ready_IVOC.reset_index(drop=True)

    # Combine them together into one dataframe    
    df_SOM_ready_IVOC = pd.concat([df_non_alkanes_IVOC, df_SOM_ready_IVOC], axis=1, sort=False)

    # Replace all the NaN with 0s
    df_SOM_ready_IVOC = df_SOM_ready_IVOC.fillna(0)

    # Give a name to the final dataframe
    df_SOM_ready_IVOC.name = str(exp_row_IVOC.loc['test_ID'].iloc[0].astype(int))
    # Save it into a file, using its name
    df_SOM_ready_IVOC.to_csv(df_SOM_ready_IVOC.name + '_IVOC' + '.csv')

# End IVOC section

# Next, VOCs

# End VOC section

In [ ]:
Do not run this cell; for reference only!
# Now we loop through all the files, through all the categories, and if the category is alkanes,
# we loop through the carbon numbers.
for i in range(len(data_files)):
    for j in range(len(categories)):
        # Check to see if you're doing alkanes. If you are, you have to loop through carbon numbers too
        if  (  (categories[j] == 'n-alkane')
             | (categories[j] == 'b-alkane')
             | (categories[j] == 'c-alkane')
            ):
            for k in range(len(carbon_num)):
                # This pulls out indices for species that match each alkane type and carbon number
                # The category matrix is missing the test_ID column though
                # , so all the indices are offset from the database columns by 1 
                temp_index_list = cat_files[i].index[ (cat_files[i]['Category']==categories[j]) 
                                                     & (cat_files[i]['Carbon number']==carbon_num[k]) ].tolist()
                # Now we find and sum up the species
                # Remember, we need the +1 in the data_files indices to account for the test_ID column
                data_index_list = [x+1 for x in temp_index_list]
                temp_conc = data_files[i].iloc[:,(data_index_list)].sum(axis=1)
                # Store it back into the dataframe
                cat_files_list[i][categories[j],carbon_num[k]] = temp_conc                
        else:            
            # This pulls out the indices for a category, in a file
            temp_index_list = cat_files[i].index[cat_files[i]['Category'] == categories[j]].tolist()
            # This pulls out, then sums, the concentrations matching the categories
            temp_conc = data_files[i].iloc[:,temp_index_list].sum(axis=1)
            # Store it in a new dataframe for categorized data
            cat_files_list[i][categories[j]] = temp_conc
            

In [ ]:
S2categorized = []
cat_files = (S2_cat)
data_files = (df_S2)
categories =  ['benzene', 'toluene', 'xylenes']
for i in range(df_S2):
    for j in range(len(categories)):         
        # This pulls out the indices for a category, in a file
        temp_index_list = cat_files[i].index[df_S2['Category'] == categories[j]].tolist()
        # Because of the test_ID column in the front, all indices should be increased by 1
        data_index_list = [x+1 for x in temp_index_list]
        # This pulls out, then sums, the concentrations matching the categories
        temp_conc = df_S2.iloc[:,data_index_list].sum(axis=1)
        # Store it in a new dataframe for categorized data
        S2categorized[i][categories[j]] = temp_conc